<a href="https://colab.research.google.com/github/l3u9/RL-mario-pytorch/blob/main/mario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
pip install gym-super-mario-bros==7.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=494879 sha256=556756d3a77931bdd4797311d3ba9ea19e05f66dd1f2b5a7fcc91e0b7adc5e41
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


In [3]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy

import gym
from gym.spaces import Box
from gym.wrappers import FrameStack

from nes_py.wrappers import JoypadSpace

import gym_super_mario_bros

강화학습의 개념

Environment: 에이전트가 상호작용하며 스스로 배우는 세계

Action a: 에이전트가 환경에 어떻게 응답하는지 행동을 나타낸다. 가능한 모든 행동의 집합을 행동 공간이라고 한다.

State s: 환경의 현재 특성 상태를 통해 나타낸다. 환경이 있을 수 있는 모든 가능한 상태 집합을 상태 공간이라고 한다.

Reward r: 포상은 환경에서 에이전트로 전달되는 피드백이다. 에이전트가 학습하고 향후 행동을 변경하도록 유도하는 것이다. 여러 시간 단계에 걸친 포상의 합을 return이라고 한다.

Action-Value-function Q'(s, a): 상태 s에서 시작하면 예상되는 리턴을 반환하고 임의의 행동 a를 선택한다. 그 다음 각각의 미래의 단계에서 포상의 합을 극대화 하는 행동을 선택하도록 한다. Q는 상태에서 행동의 "품질"을 나타낸다.

In [15]:
# Environment 초기화

if gym.__version__ < '0.26':
  env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', new_step_api=True)
else:
  env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0', render_mode='rgb', apply_api_compatibility=True)

env = JoypadSpace(env, [["right"], ["right", 'A']])

env.reset()
next_state, reward, done, trunc, info = env.step(action=0)
print(f"{next_state.shape}, \n {reward}, \n {done}, \n {info}")



(240, 256, 3), 
 0.0, 
 False, 
 {'coins': 0, 'flag_get': False, 'life': 2, 'score': 0, 'stage': 1, 'status': 'small', 'time': 400, 'world': 1, 'x_pos': 40, 'y_pos': 79}


In [16]:
# 전처리
class SkipFrame(gym.Wrapper):
  def __init__(self, env, skip):
    super().__init__(env)
    self._skip = skip

  def step(self, action):
    total_reward = 0.0
    for i in range(self.skip):
      obs, reward, done, trunk, info = self.env.step(action)
      total_reward += reward
      if done:
        break
    return obs, total_reward, done, trunk, info


class GrayScaleObservation(gym.ObservationWrapper):
  def __init__(self, env):
    super().__init__(env)
    obs_shape = self.observation_space.shape[:2]
    self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

  def permute_orientation(self, observation):
    observation = np.transpose(observation, (2, 0, 1))
    observation = torch.tensor(observation.copy(), dtype=torch.float)
    return observation

  def observation(self, observation):
    observation = self.permute_orientation(observation)
    transform = T.Grayscale()
    observation = transform(observation)
    return observation

class ResizeObservation(gym.ObservationWrapper):
  def __init__(self, env, shape):
    super().__init__(env)
    if isinstance(shape, int):
      self.shape = (shape, shape)
    else:
      self.shape = tuple(shape)

    obs_shape = self.shape + self.observation_space.shape[2:]

    self.observation_space = Box(low=0, high=255, shape=obs_shape, dtype=np.uint8)

  def observation(self, observation):
    transforms = T.Compose([T.Resize(self.shape), T.Normalize(0, 255)])
    observation = transforms(observation).squeeze(0)
    return observation


env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)

if gym.__version__ < '0.26':
  env = FrameStack(env, num_stack=4, new_step_api=True)
else:
  env = FrameStack(env, num_stack=4)

